In [ ]:
print("Hello")

## Conversational App for Itinerary Planning

In [ ]:
import os
from dotenv import load_dotenv
#from langchain import HuggingFaceHub

from langchain import PromptTemplate, LLMChain

load_dotenv()

#os.environ["HUGGINGFACEHUB_API_TOKEN"]
#retrieving api key
key=os.environ['OPENAI_API_KEY']

In [ ]:
from langchain.prompts import (
    ChatPromptTemplate,
    MessagePlaceHolder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

from langchain.chains import LLMChain, ConversationChain
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI()

### Sample bot with no memory

In [ ]:
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

from langchain.chains import LLMChain, ConversationChain
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI()
messages = [
    SystemMessage(content="You are a helpful assistant that helps the user to plan an optimized itinerary."),
    HumanMessage(content="I'm going to Rome for 2 days, what can I visit?")
]
chat(messages)

In [ ]:
output = chat(messages)
print(output.content)

### Adding Memory

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=chat, verbose=True, memory=memory
)

conversation.run("Hi there!")

In [ ]:
conversation.run("what is the most iconic place in Rome?")

In [ ]:
conversation.run("What kind of other events?")

In [ ]:
memory.load_memory_variables({})

In [ ]:
from langchain.prompts import (
    ChatPromptTemplate,
    MessagePlaceHolder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.memory import ConversationBufferMemory

#LLM
repo_id = 'databricks/dolly-v2-3b'
llm = HuggingFaceHub(
    repo_id = repo_id, model_kwargs={"temperature":0.5, "max_length":1000}
)

#Prompt
prompt = ChatPromptTemplate.from_messages(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "You are a helpful assistant that help the user to plan an optimized itinerary."
        ),
        # The`variable_name` here is what must align with memory
        MessagePlaceHolder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)

# Notice that we `return_messages=True` to fit into the MessagesPlaceholder
# Notice that `"chat_history"` aligns with the MessagesPlaceholder name
memory = ConversationBufferMemory(memory_key="chat_history", return_message=True)
conversation = LLMChain(
    llm=chat,
    prompt=prompt,
    verbose=False,
    memory=memory
)

In [ ]:
while True:
    query = input('you: ')
    if query == 'q':
        break
    output = conversation({"'question": query})
    print('User ', query)
    print('AI System: ', output['text'])

### Adding non parametric knowledge

In [ ]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader

import os

from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_API_KEY"]

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 200
)

raw_documents = PyPDFLoader('italy_travel.pdf').load()
documents = text_splitter.split_documents(raw_documents)
db = FAISS.from_documents(documents, OpenAIEmbeddings())

In [ ]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key= 'chat_history',
    return_messages=True
)

llm = ChatOpenAI()
qa_chain = ConversationalRetrievalChain.from_llm(llm,  retriever=db.as_retriever(), memory=memory, verbose=True)
qa_chain.run({'question':'Give me some review about the Pantheon'})

In [ ]:
from langchain.prompts.prompt import PromptTemplate

custom_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone quuestion.
If you cannot find the answer in the document provided, ignore the document and answer anyway.
Chat History:
{chat_history}
Follow up Input: {question}
Standalone question:"""

CUSTOM_QUESTION_PROMPT = PromptTemplate.from_template(custom_template)

memory = ConversationBufferMemory(
    memory_key='chat_history',
    return_messages=True
)

qa_chain = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=db.as_retriever(),
    condense_question_prompt = CUSTOM_QUESTION_PROMPT,
    memory = memory,
    verbose=True
)

qa_chain.run({'question':'What can I visit in India?'})

### Agent with memory for conversations

In [ ]:
from langchain.agents.agent_toolkits import create_retriever_tool

tool = create_retriever_tool(
    db.as_retriever(),
    "italy_travel",
    "Searches and returns documents regarding Italy"
)

tools = [tool]

memory = ConversationBufferMemory(
    memory_key='chat_history',
    return_messages=True
)

from langchain.agents.agent_toolkits import create_conversational_retrieval_agent

from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature=0)

agent_executor = create_conversational_retrieval_agent(llm, tools, memory_key='chat_history', verbose=True)

In [ ]:
agent_executor({'input':"hi, I'm Vale"})

In [ ]:
agent_executor({"input":"Tell me something about Pantheon"})

In [ ]:
output = agent_executor({"input":"What can I visit in India in 3 days?"})
output['output']

### Adding external tools

In [ ]:
from langchain import SerpAPIWrapper
from langchain.agents import AgentType, initialize_agent
from langchain.llms import OpenAI
from langchain.tools imporort BaseTool, StructureTool, Tool, tool

import os
from dotenv import load_dotenv

load_dotenv()

key = os.environ["SERPAPI_API_KEY"]

search = SerpAPIWrapper()

In [ ]:
tools = [
    Tool.from_function(
        func=search.run,
        name="Search",
        description="useful for when you need to answer questions about current events"
    ),
    create_retriever_tool(
        db.as_retriever(),
        "italy_travel",
        "Searches and returns documents regarding Italy."
    )
]

agent_executor = create_conversational_retrieval_agent(llm, tools, memory_key="chat_history", verbose=True)

In [ ]:
memory

In [ ]:
agent_executor({"input":"What can I visit in India in 3 days?"})

In [ ]:
agent_executor({"input": "What is the current weather in Delhi?"})

In [ ]:
agent_executor({"input":"I'm travelling to Italy, can you give me some suggestions of the main attractions?"})